# Getting started with Cosmos notebooks
In this notebook, we'll learn how to use Cosmos notebook features. We'll create a database and container, import some sample data in a container in Azure Cosmos DB and run some queries over it.

### Create new database and container

To connect to the service, you can use our built-in instance of ```cosmos_client```. This is a ready to use instance of [CosmosClient](https://docs.microsoft.com/python/api/azure-cosmos/azure.cosmos.cosmos_client.cosmosclient?view=azure-python) from our Python SDK. It already has the context of this account baked in. We'll use ```cosmos_client``` to create a new database called **RetailDemo** and container called **WebsiteData**.

Our dataset will contain events that occurred on the website - e.g. a user viewing an item, adding it to their cart, or purchasing it. We will partition by CartId, which represents the individual cart of each user. This will give us an even distribution of throughput and storage in our container. Learn more about how to [choose a good partition key.](https://docs.microsoft.com/azure/cosmos-db/partition-data)

In [4]:
import azure.cosmos
from azure.cosmos.partition_key import PartitionKey

database = cosmos_client.create_database_if_not_exists('RetailDemo')
print('Database RetailDemo created')

container = database.create_container_if_not_exists(id='WebsiteData', partition_key=PartitionKey(path='/CartID'))
print('Container WebsiteData created')


Database RetailDemo created
Container WebsiteData created


#### Set the default database and container context to the new resources

We can use the ```%database {database_id}``` and ```%container {container_id}``` syntax.

In [5]:
%database RetailDemo

In [6]:
%container WebsiteData

### Load in sample JSON data and insert into the container. 
We'll use the **UpsertItem** operation to create the item if it doesn't exist, or replace it if it already exists. 

In [7]:
# We can install custom packages using pip install
import sys
!{sys.executable} -m pip install progressbar2 --user

In [8]:
## Read data from storage
import urllib.request, json 
import progressbar

with urllib.request.urlopen("https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/websiteData-small.json") as url:
    data = json.loads(url.read().decode())

for event in progressbar.progressbar(data):
    try: 
        container.upsert_item(body=event)
    except errors.CosmosError as e:
        raise

N/A% (0 of 7) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--
 42% (3 of 7) |###########               | Elapsed Time: 0:00:00 ETA:  00:00:00
100% (7 of 7) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


The new database and container should show up under the **Data** section. Use the refresh icon after completing the previous cell. 

![RefreshData](https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/refreshData.png)


### Run a query using the built-in Azure Cosmos notebook magic


In [28]:
%%sql
SELECT c.Action, c.Price as ItemRevenue, c.Country, c.Item FROM c

We can get more information about the %%sql command using ```%%sql?```

### Next steps

Now that you've learned how to use basic notebook functionality, follow the **Visualization.ipynb** notebook to further analyze and visualize our data. You can find it under the **Sample Notebooks** section.